# 当当网商品采集
地址：https://search.dangdang.com/?key=java&act=input&page_index=1
采集信息： 商品名称  描述  价格  评分  评论  图片

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait   # 等待页面元素加载
from selenium.webdriver.support import expected_conditions # 判断模块
import pandas,os


class App:
    """初始化方法"""
    def __init__(self):
        # 定义url地址
        self.url="https://search.dangdang.com/?key={key}&act=input&page_index=1"
        # 搜索框内容
        self.key="java"
        # 获取浏览器对象
        self.web=webdriver.Chrome()
        pass


    """获取数据"""
    def getData(self):
        self.web.get(self.url.format(key=self.key))
        # 隐式等待，如果访问页面数据过多，等待加载数据
        self.web.implicitly_wait(3)
        try:
            for i in range(0,10):
                print("开始采集第{page}页数据".format(page=i+1))
                # 采集商品的容器标签li
                # 等待加载+查询标签:判断标签是否存在，等到该标签加载再继续执行下一步
                WebDriverWait(self.web,timeout=10).until(expected_conditions.presence_of_element_located((By.ID,'component_59')))
                # 解析数据
                goods=self.parseData(self.web)
                print(goods)
                # 存储数据
                self.saveData(goods,i)
                # 下一页
                WebDriverWait(self.web,timeout=10).until(expected_conditions.presence_of_element_located((By.LINK_TEXT,'下一页')))
                # 点击下一页
                self.web.find_element(By.LINK_TEXT,'下一页').click()
                print("已经点击下一页,当前是第{page}页".format(page=(i+1)))
                pass
        except Exception as e:
            print("异常",e)
        finally:
            self.web.close() # 关闭浏览器     
        pass
    

    """存储数据"""
    def saveData(self,goods,page):
        dowload="./当当网商品数据/"
        if not os.path.exists(dowload): # 判断文件夹是否存在
            # 创建文件夹
            os.mkdir(dowload)
        gooddic={
            "商品名称":[good[0]  for good in goods],
            "商品价格":[good[2]  for good in goods],
            "商品评分":[good[3]  for good in goods],
            "商品评论":[good[4]  for good in goods],
            "商品图片":[good[5]  for good in goods],
            "商品详情":[good[1]  for good in goods],
        }
        pandas.DataFrame(data=gooddic,columns=['商品名称','商品价格','商品评分','商品图片','商品详情']).to_csv(dowload+"第{page}页商品信息.csv".format(page=page+1),encoding="utf-8",index=False)
    

    """解析数据"""
    def parseData(self,web):
        goods=[]  # 存储采集数据
        li_list=web.find_element(By.ID,"component_59").find_elements(By.TAG_NAME,"li")
        for li in li_list:
            # 数据
            good_name=li.find_element(By.CLASS_NAME,"name").text
            good_detail=li.find_element(By.CLASS_NAME,"detail").text
            search_now_price=li.find_element(By.CLASS_NAME,"search_now_price").text
            # 判断查询标签返回列表长度
            if len(li.find_elements(By.CSS_SELECTOR,'.search_star_black'))>0:
                # 评分
                good_score=li.find_element(By.CSS_SELECTOR,".search_star_black span").get_attribute("style")[7:9]
                # 评论
                good_commit=li.find_element(By.CLASS_NAME,"search_comment_num").text
            else:
                good_score=0
                good_commit=0
            # 图片
            good_pic=li.find_element(By.TAG_NAME,"img").get_attribute("src")
            # 将所有数据添加goods中
            goods.append((good_name,good_detail,search_now_price,good_score,good_commit,good_pic))
        return goods;

    
    def main(self):
        self.getData()
        pass

App().main()